In [3]:
# Import all the required libraries for data retrieval and visualization

import requests as req
from pprint import pprint as print
from datetime import datetime
from datetime import timedelta
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
import numpy as np
from math import radians

In [4]:
#All API Urls
sensors_url  = 'https://api.sealevelsensors.org/v1.0/Sensors'
properties_url = 'https://api.sealevelsensors.org/v1.0/ObservedProperties'
observations_url ='https://api.sealevelsensors.org/v1.0/Observations'
base_url_sls  = 'https://api.sealevelsensors.org/v1.0/Things'

API types include the /Sensors and /ObservedProperties, both of which are metadata, and then real time observations are updated in /Observations

In [20]:
session = req.Session()

def get_sensor_datastreams():
    """
    Creates a list of all sensors with datastream links.
    Returns:
        sensors (list): a list of 'sensors', each sensor being a dictonary with information on the sensor
    """
    api_response = session.get(base_url_sls).json()
    def create_sensor_obj(sensor):
        """Turns a 'sensor' from the API into a succinct dictionary."""
        location = (session.get(sensor["Locations@iot.navigationLink"])).json()["value"][0]
        historical_location = (session.get(sensor["HistoricalLocations@iot.navigationLink"])).json()["value"][0]
        coordinates = location["location"]["coordinates"]
        return {
          "Name":   sensor["name"],
          "ID": sensor["@iot.id"],
          "Status": sensor["properties"].get("status"),
          "Inundation Types": sensor["properties"].get("inundationTypes"),
          "Installation Type": sensor["properties"].get("installationType"),
          "Date of Last Status Change": sensor["properties"].get("dateOfLastStatusChange"),
          "Description":   sensor["description"],
          "Datastreams link":   sensor["Datastreams@iot.navigationLink"],
          "Location": location["location"]
          "Coordinates": location["location"]["coordinates"],
          "Sensor Installed": historical_location["time"],
          "Elevation":   sensor["properties"].get("elevationNAVD88"),
          }
    return list(map(create_sensor_obj, api_response["value"]))

In [23]:
#things_data is a list of dictionaries where each dictionary contains information about...
#...a given thing
things_data = get_sensor_datastreams()
things_data

[{'name': 'gt-envsense-005',
  'id': 2,
  'status': 'outOfService',
  'Inundation Types': 'coastal,fluvial',
  'Installation Type': 'bridge',
  'Date of Last Status Change': '2024-03-18',
  'desc': 'Hutchinson Island Water Level Monitor',
  'Datastreams link': 'https://api.sealevelsensors.org/v1.0/Things(2)/Datastreams',
  'Historical Locations link': 'https://api.sealevelsensors.org/v1.0/Things(2)/HistoricalLocations',
  'Locations link': 'https://api.sealevelsensors.org/v1.0/Things(2)/Locations',
  'elev': '4.711',
  'coords': [-81.084099, 32.085707]},
 {'name': 'gt-envsense-004',
  'id': 3,
  'status': 'active',
  'Inundation Types': 'coastal,fluvial',
  'Installation Type': 'dock',
  'Date of Last Status Change': '2024-03-18',
  'desc': 'Kilkenny Creek Water Level Monitor',
  'Datastreams link': 'https://api.sealevelsensors.org/v1.0/Things(3)/Datastreams',
  'Historical Locations link': 'https://api.sealevelsensors.org/v1.0/Things(3)/HistoricalLocations',
  'Locations link': 'https

In [26]:
#Filter our things that are not water sensors
for dict in things_data:
    if not "Water Level" in dict.get('desc'):
        things_data.remove(dict)

In [30]:
df = pd.DataFrame(things_data)
df

,name,id,status,Inundation Types,Installation Type,Date of Last Status Change,desc,Datastreams link,Historical Locations link,Locations link,elev,coords
0,gt-envsense-005,2,outOfService,"coastal,fluvial",bridge,2024-03-18,Hutchinson Island Water Level Monitor,https://api.sealevelsensors.org/v1.0/Things(2)...,https://api.sealevelsensors.org/v1.0/Things(2)...,https://api.sealevelsensors.org/v1.0/Things(2)...,4.711,"[-81.084099, 32.085707]"
1,gt-envsense-004,3,active,"coastal,fluvial",dock,2024-03-18,Kilkenny Creek Water Level Monitor,https://api.sealevelsensors.org/v1.0/Things(3)...,https://api.sealevelsensors.org/v1.0/Things(3)...,https://api.sealevelsensors.org/v1.0/Things(3)...,2.7,"[-81.201916, 31.791509]"
2,gt-envsense-011,5,active,"coastal,fluvial",dock,2024-03-18,Bull River Marina Water Level Monitor,https://api.sealevelsensors.org/v1.0/Things(5)...,https://api.sealevelsensors.org/v1.0/Things(5)...,https://api.sealevelsensors.org/v1.0/Things(5)...,3.041,"[-80.959417, 32.034717]"
3,gt-envsense-057,7,active,"coastal,fluvial",dock,2024-03-18,Coffee Bluff Marina Water Level Monitor,https://api.sealevelsensors.org/v1.0/Things(7)...,https://api.sealevelsensors.org/v1.0/Things(7)...,https://api.sealevelsensors.org/v1.0/Things(7)...,2.536,"[-81.153906, 31.935783]"
4,gt-sentrius-011,9,None,None,None,None,Garden City City Hall Environmental Monitor,https://api.sealevelsensors.org/v1.0/Things(9)...,https://api.sealevelsensors.org/v1.0/Things(9)...,https://api.sealevelsensors.org/v1.0/Things(9)...,None,"[-81.20544, 32.063457]"
5,gt-envsense-009,14,active,"coastal,fluvial",bridge,2024-03-18,Walthour Road Water Level Monitor,https://api.sealevelsensors.org/v1.0/Things(14...,https://api.sealevelsensors.org/v1.0/Things(14...,https://api.sealevelsensors.org/v1.0/Things(14...,2.768,"[-80.962163, 32.018171]"
6,gt-envsense-003,15,outOfService,"coastal,fluvial",bridge,2023-04-14T16:40:22.774Z,Oatland Island Road Water Level Monitor,https://api.sealevelsensors.org/v1.0/Things(15...,https://api.sealevelsensors.org/v1.0/Things(15...,https://api.sealevelsensors.org/v1.0/Things(15...,2.736,"[-81.011631, 32.052507]"
7,gt-envsense-096,48,active,"coastal,fluvial",dock,2024-03-18,Hwy 17 at Salt Creek Water Level Monitor,https://api.sealevelsensors.org/v1.0/Things(48...,https://api.sealevelsensors.org/v1.0/Things(48...,https://api.sealevelsensors.org/v1.0/Things(48...,2.74,"[-81.203851, 32.039975]"
8,gt-envsense-013,49,active,"coastal,fluvial",bridge,2023-12-05T12:00:00.000Z,Hunt Drive on Burnside Island Water Level Monitor,https://api.sealevelsensors.org/v1.0/Things(49...,https://api.sealevelsensors.org/v1.0/Things(49...,https://api.sealevelsensors.org/v1.0/Things(49...,3.512,"[-81.086168, 31.928811]"
9,gt-envsense-037,51,active,"coastal,fluvial",bridge,2023-12-05T12:00:00.000Z,Catalina Drive Water Level Monitor,https://api.sealevelsensors.org/v1.0/Things(51...,https://api.sealevelsensors.org/v1.0/Things(51...,https://api.sealevelsensors.org/v1.0/Things(51...,2.805,"[-80.866956, 32.006419]"
